In [1]:
# basics
!pip -q install rosbags open3d opencv-python-headless numpy pyyaml tqdm ultralytics

# (optional) speedups
# import os; os.environ["OMP_NUM_THREADS"] = "1"

# !pip install rosbags ultralytics opencv-python numpy PyYAML open3d tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.0 MB/s eta 0:00:00


In [7]:
pip install open3d -q


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Point these to your actual paths on Drive:
!cp "/content/drive/MyDrive/Perceptra/scripts/list_bag_topics.py" /content/
!cp "/content/drive/MyDrive/Perceptra/scripts/colorize_and_merge.py" /content/


Mounted at /content/drive


In [3]:
import sqlite3, os, cv2, numpy as np
from pathlib import Path
from PIL import Image
import base64

def extract_rgb_frames(bag_dir, out_dir, topic_hint='color', max_frames=50):
    bag_path = Path(bag_dir)
    db3_files = list(bag_path.glob('*.db3'))
    if not db3_files:
        raise FileNotFoundError(f"No .db3 found under {bag_dir}")
    conn = sqlite3.connect(str(db3_files[0]))
    c = conn.cursor()

    # list all topics
    topics = [row[0] for row in c.execute('SELECT name FROM topics')]
    print("Topics:", topics)

    # pick the first one containing 'image'
    topic = [t for t in topics if 'image' in t and topic_hint in t]
    if not topic:
        topic = [t for t in topics if 'image' in t]
    topic = topic[0]
    print("Using topic:", topic)

    # get topic_id
    tid = c.execute("SELECT id FROM topics WHERE name=?", (topic,)).fetchone()[0]
    msgs = c.execute("SELECT data FROM messages WHERE topic_id=?", (tid,)).fetchmany(max_frames)

    os.makedirs(out_dir, exist_ok=True)
    for i,(data,) in enumerate(msgs):
        # raw image bytes are serialized as sensor_msgs/Image in ROS2 (binary blob)
        # We'll use OpenCV imdecode assuming JPEG encoding
        # Find JPEG start marker (FF D8)
        idx = data.find(b'\xff\xd8')
        if idx >= 0:
            img_bytes = data[idx:]
            img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
            if img is not None:
                out = Path(out_dir)/f"frame_{i:03d}.jpg"
                cv2.imwrite(str(out), img)
    conn.close()
    print(f"[OK] Saved frames to {out_dir}")


In [4]:
extract_rgb_frames(
    bag_dir="/content/drive/MyDrive/Perceptra/Challenge Surveys/office/rosbag2_2025_10_20-16_09_39",
    out_dir="/content/drive/MyDrive/Perceptra/frames/office",
    max_frames=30
)

extract_rgb_frames(
    bag_dir="/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/rosbag2_2025_10_20-16_47_22",
    out_dir="/content/drive/MyDrive/Perceptra/frames/bathroom",
    max_frames=30
)


Topics: ['/imu', '/odom', '/livox/imu', '/livox/lidar', '/zed/zed_node/depth/depth_registered/compressedDepth', '/zed/zed_node/odom', '/scan', '/tf', '/zed/zed_node/rgb/image_rect_color/compressed', '/tf_static', '/zed/zed_node/rgb/camera_info']
Using topic: /zed/zed_node/rgb/image_rect_color/compressed
[OK] Saved frames to /content/drive/MyDrive/Perceptra/frames/office
Topics: ['/imu', '/odom', '/livox/imu', '/livox/lidar', '/scan', '/tf', '/tf_static', '/zed/zed_node/depth/depth_registered/compressedDepth', '/zed/zed_node/odom', '/zed/zed_node/rgb/image_rect_color/compressed', '/zed/zed_node/rgb/camera_info']
Using topic: /zed/zed_node/rgb/image_rect_color/compressed
[OK] Saved frames to /content/drive/MyDrive/Perceptra/frames/bathroom


In [5]:
# # Sanity Check
# !yolo detect predict model=yolov8l.pt source="/content/drive/MyDrive/Perceptra/frames/office" conf=0.4 save=True


In [6]:
!python /content/drive/MyDrive/Perceptra/scripts/detect_and_overlay.py \
  --room-yaml "/content/drive/MyDrive/Perceptra/Challenge Surveys/office/room.yaml" \
  --room-pgm  "/content/drive/MyDrive/Perceptra/Challenge Surveys/office/room.pgm" \
  --rgb-dir   "/content/drive/MyDrive/Perceptra/frames/office" \
  --out-dir   results/office \
  --model yolov8n.pt \
  --only-best-frame



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[INFO] Using best frame frame_019.jpg
[OK] Saved 2 detections → results/office/detections.json
[OK] Saved overlay → results/office/map_with_detections.png


In [7]:
!python /content/drive/MyDrive/Perceptra/scripts/detect_and_overlay.py \
  --room-yaml "/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/room.yaml" \
  --room-pgm  "/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/room.pgm" \
  --rgb-dir   "/content/drive/MyDrive/Perceptra/frames/bathroom" \
  --out-dir   results/bathroom \
  --model yolov8n.pt \
  --only-best-frame


[OK] Saved 0 detections → results/bathroom/detections.json
[OK] Saved overlay → results/bathroom/map_with_detections.png


In [19]:
BAG = "/content/drive/MyDrive/Perceptra/Challenge Surveys/office/rosbag2_2025_10_20-16_09_39"

!python /content/list_bag_topics.py --bags "$BAG"


== Available topics ==
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/lidar                                   sensor_msgs/msg/PointCloud2
/livox/lidar                                   sensor_msgs/msg/PointCloud2
/livox/lidar           

In [28]:
!python /content/drive/MyDrive/Perceptra/scripts/colorize_and_merge.py \
  --bags "{BAG}" \
  --cloud-topic "/livox/lidar" \
  --image-topic "/zed/zed_node/rgb/image_rect_color/compressed" \
  --caminfo-topic "/zed/zed_node/rgb/camera_info" \
  --tf-topics /tf /tf_static \
  --world-frame base_link \
  --sync-tol 0.5 \
  --stride 6 \
  --max-clouds 600 \
  --voxel 0.15 \
  --verbose \
  --out /content/office_colored.ply


[INFO] clouds: 100  images: 3585  camera frames: ['zed_left_camera_optical_frame']
Colorizing & merging clouds:   0% 0/100 [00:00<?, ?it/s][DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame=

In [30]:
BAG = "/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/rosbag2_2025_10_20-16_47_22"

!python /content/list_bag_topics.py --bags "$BAG"


== Available topics ==
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/imu                                           sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/imu                                     sensor_msgs/msg/Imu
/livox/lidar                                   sensor_msgs/msg/PointCloud2
/livox/lidar                                   sensor_msgs/msg/PointCloud2
/livox/lidar                                   sensor_msgs/msg/PointCloud2
/odom                                          nav_msgs/msg/Odometry
/odom                                          nav_msgs/msg/Odometry
/odom                                          nav_msgs/msg/Odometry
/scan                                          sensor_msgs/msg/LaserScan
/scan                                          sensor_msgs/msg/LaserSc

In [31]:
!python /content/drive/MyDrive/Perceptra/scripts/colorize_and_merge.py \
  --bags "{BAG}" \
  --cloud-topic "/livox/lidar" \
  --image-topic "/zed/zed_node/rgb/image_rect_color/compressed" \
  --caminfo-topic "/zed/zed_node/rgb/camera_info" \
  --tf-topics /tf /tf_static \
  --world-frame base_link \
  --sync-tol 0.5 \
  --stride 6 \
  --max-clouds 600 \
  --voxel 0.15 \
  --verbose \
  --out /content/bathroom_colored.ply


[INFO] clouds: 100  images: 1347  camera frames: ['zed_left_camera_optical_frame']
Colorizing & merging clouds:   0% 0/100 [00:00<?, ?it/s][DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame='zed_left_camera_optical_frame'  K=ok  TF_img=ok
[DBG] img_frame=

In [32]:
!python /content/drive/MyDrive/Perceptra/test/verify_ch2.py --ply /content/office_colored.ply /content/bathroom_colored.ply --out ch2_verify_results


error: XDG_RUNTIME_DIR not set in the environment.
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[OK] Wrote report to ch2_verify_results/report.json
- /content/office_colored.ply: 72369 pts | colored: 24.3% | extent (m): [21.538 22.33   4.47 ]
  proj preview: ch2_verify_results/office_colored_proj.png
- /content/bathroom_colored.ply: 13312 pts | colored: 21.2% | extent (m): [6.379 9.132 2.749]
  proj preview: ch2_verify_results/bathroom_colored_proj.png
